## Install and load necessary libraries.

In [ ]:
%pip install tableone

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.6/41.6 kB 597.0 kB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tableone

from google.colab import files
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

%load_ext google.colab.data_table

PROJECT_ID = 'ID'

%env GOOGLE_CLOUD_PROJECT = PROJECT_ID

Authenticated
env: GOOGLE_CLOUD_PROJECT=PROJECT_ID


### CRRT

In [ ]:
%%bigquery crrt_patients_with_icud --project PROJECT_ID

WITH first_admission AS (
  SELECT
    *,
    ROW_NUMBER() OVER (PARTITION BY subject_id ORDER BY admittime) AS rn
  FROM
    `physionet-data.mimiciv_hosp.admissions`
),
first_sofa AS (
  SELECT
    *,
    ROW_NUMBER() OVER (PARTITION BY stay_id ORDER BY hr) AS rn
  FROM
    `physionet-data.mimiciv_derived.sofa`
),
first_sapsii AS (
  SELECT
    *,
    ROW_NUMBER() OVER (PARTITION BY stay_id) AS rn
  FROM
    `physionet-data.mimiciv_derived.sapsii`
),
first_charlson AS (
  SELECT
    *,
    ROW_NUMBER() OVER (PARTITION BY subject_id) AS rn
  FROM
    `physionet-data.mimiciv_derived.charlson`
),
first_mech AS (
  SELECT
    *,
    ROW_NUMBER() OVER (PARTITION BY stay_id) AS rn
  FROM
    `physionet-data.mimiciv_derived.ventilation`
)

SELECT
  crrt.*,
  icud.*,
  a.admittime,
  a.hadm_id,
  a.dischtime,
  a.insurance,
  a.language,
  a.marital_status,
  a.race,
  a.admission_type,
  a.hospital_expire_flag,
  sofa.sofa_24hours,
  saps.sapsii,
  ch.charlson_comorbidity_index,
  mech.ventilation_status,
  dia.icd_code
FROM
  `physionet-data.mimiciv_derived.crrt` AS crrt
JOIN
  `physionet-data.mimiciv_derived.icustay_detail` AS icud
ON
  crrt.stay_id = icud.stay_id
JOIN
  (
    SELECT
      *
    FROM
      first_admission
    WHERE
      rn = 1
  ) AS a
ON
  icud.subject_id = a.subject_id
JOIN
  (
    SELECT
      *
    FROM
      first_sofa
    WHERE
      rn = 1
  ) AS sofa
ON
  crrt.stay_id = sofa.stay_id
JOIN
  (
    SELECT
      *
    FROM
      first_sapsii
    WHERE
      rn = 1
  ) AS saps
ON
  crrt.stay_id = saps.stay_id
JOIN
  (
    SELECT
      *
    FROM
      first_charlson
    WHERE
      rn = 1
  ) AS ch
ON
  icud.subject_id = ch.subject_id
JOIN
  (
    SELECT
      *
    FROM
      first_mech
    WHERE
      rn = 1
  ) AS mech
ON
  crrt.stay_id = mech.stay_id
JOIN
  `physionet-data.mimiciv_hosp.diagnoses_icd` AS dia
ON
  a.hadm_id = dia.hadm_id;


Query is running:   0%|          |

Downloading:   0%|          |

In [ ]:
# Assuming your dataframe is named df and the column with race categories is named 'race'
race_mapping = {
    'WHITE': 'White',
    'WHITE - OTHER EUROPEAN': 'White',
    'PORTUGUESE': 'White',
    'WHITE - RUSSIAN': 'White',
    'WHITE - BRAZILIAN': 'White',
    'WHITE - EASTERN EUROPEAN': 'White',
    'BLACK/AFRICAN AMERICAN': 'Black/African American',
    'BLACK/CARIBBEAN ISLAND': 'Black/African American',
    'BLACK/AFRICAN': 'Black/African American',
    'BLACK/CAPE VERDEAN': 'Black/African American',
    'HISPANIC OR LATINO': 'Hispanic/Latino',
    'HISPANIC/LATINO - PUERTO RICAN': 'Hispanic/Latino',
    'HISPANIC/LATINO - DOMINICAN': 'Hispanic/Latino',
    'HISPANIC/LATINO - GUATEMALAN': 'Hispanic/Latino',
    'HISPANIC/LATINO - COLUMBIAN': 'Hispanic/Latino',
    'HISPANIC/LATINO - MEXICAN': 'Hispanic/Latino',
    'HISPANIC/LATINO - HONDURAN': 'Hispanic/Latino',
    'HISPANIC/LATINO - SALVADORAN': 'Hispanic/Latino',
    'HISPANIC/LATINO - CUBAN': 'Hispanic/Latino',
    'ASIAN': 'Asian',
    'ASIAN - SOUTH EAST ASIAN': 'Asian',
    'ASIAN - CHINESE': 'Asian',
    'ASIAN - ASIAN INDIAN': 'Asian',
    'NATIVE HAWAIIAN OR OTHER PACIFIC ISLANDER': 'Native Hawaiian or Other Pacific Islander',
    'AMERICAN INDIAN/ALASKA NATIVE': 'American Indian/Alaska Native',
    'OTHER': 'Other',
    'MULTIPLE RACE/ETHNICITY': 'Other',
    'UNKNOWN': 'Unknown/Declined to Answer',
    'UNABLE TO OBTAIN': 'Unknown/Declined to Answer',
    'PATIENT DECLINED TO ANSWER': 'Unknown/Declined to Answer'
}
crrt_patients_with_icud_mapped = crrt_patients_with_icud
# Apply the mapping to the 'race' column
crrt_patients_with_icud_mapped['race'] = crrt_patients_with_icud['race'].map(race_mapping)

In [ ]:
tabel = tableone.tableone(
    crrt_patients_with_icud_mapped.groupby('stay_id').first(),
    columns = [
        'gender',
        'insurance',
        'language',
        'marital_status',
        'race',
        'admission_type',
        'hospital_expire_flag',
        'los_icu',
        'admission_age',
        'sofa_24hours',
        'ventilation_status',
        'sapsii',
        ],
    categorical = [
        'gender',
        'insurance',
        'language',
        'marital_status',
        'race',
        'admission_type',
        'ventilation_status',
        ],
    groupby= 'hospital_expire_flag'
)

In [ ]:
tabel

Grouped by hospital_expire_flag                                       
                                                                                           Missing      Overall            0            1
n                                                                                                          2454         1416         1038
gender, n (%)            F                                                                           958 (39.0)   547 (38.6)   411 (39.6)
                         M                                                                          1496 (61.0)   869 (61.4)   627 (60.4)
insurance, n (%)         Medicaid                                                                     221 (9.0)    137 (9.7)     84 (8.1)
                         Medicare                                                                   1015 (41.4)   569 (40.2)   446 (43.0)
                         Other                                                                      1218 (49.6)   710 (50.1)   508 (48.9)
language, n (%)          ?                                                                           266 (10.8)   151 (10.7)   115 (11.1)
                         ENGLISH                                                                    2188 (89.2)  1265 (89.3)   923 (88.9)
marital_status, n (%)    DIVORCED                                                                     176 (7.2)    115 (8.1)     61 (5.9)
                         MARRIED                                                                    1123 (45.8)   649 (45.8)   474 (45.7)
                         None                                                                        262 (10.7)     87 (6.1)   175 (16.9)
                         SINGLE                                                                      701 (28.6)   448 (31.6)   253 (24.4)
                         WIDOWED                                                                      192 (7.8)    117 (8.3)     75 (7.2)
race, n (%)              American Indian/Alaska Native                                                 10 (0.4)      8 (0.6)      2 (0.2)
                         Asian                                                                         66 (2.7)     43 (3.0)     23 (2.2)
                         Black/African American                                                      376 (15.3)   266 (18.8)   110 (10.6)
                         Hispanic/Latino                                                              106 (4.3)     65 (4.6)     41 (3.9)
                         Native Hawaiian or Other Pacific Islander                                      6 (0.2)      3 (0.2)      3 (0.3)
                         None                                                                           1 (0.0)      1 (0.1)             
                         Other                                                                         69 (2.8)     43 (3.0)     26 (2.5)
                         Unknown/Declined to Answer                                                  364 (14.8)    126 (8.9)   238 (22.9)
                         White                                                                      1456 (59.3)   861 (60.8)   595 (57.3)
admission_type, n (%)    AMBULATORY OBSERVATION                                                        45 (1.8)     28 (2.0)     17 (1.6)
                         DIRECT EMER.                                                                 168 (6.8)    115 (8.1)     53 (5.1)
                         DIRECT OBSERVATION                                                            50 (2.0)     35 (2.5)     15 (1.4)
                         ELECTIVE                                                                      85 (3.5)     57 (4.0)     28 (2.7)
                         EU OBSERVATION                                                                91 (3.7)     55 (3.9)     36 (3.5)
                         EW EMER.                                

In [ ]:
patients = crrt_patients_with_icud_mapped[['stay_id','gender',
        'insurance',
        'language',
        'marital_status',
        'race',
        'admission_type',
        'hospital_expire_flag',
        'los_icu',
        'admission_age',
        'sofa_24hours','sapsii','charlson_comorbidity_index','icd_code']].groupby('stay_id').first().drop_duplicates()

In [ ]:
crrt_patients_with_icud.groupby('stay_id').first()

,charttime,crrt_mode,access_pressure,blood_flow,citrate,current_goal,dialysate_fluid,dialysate_rate,effluent_pressure,filter_pressure,...,language,marital_status,race_1,admission_type,hospital_expire_flag_1,sofa_24hours,sapsii,charlson_comorbidity_index,ventilation_status,icd_code
stay_id,,,,,,,,,,,,,,,,,,,,,
30003226,2123-02-27 09:00:00,CVVHDF,-46.0,120.0,180.0,0.0,Prismasate K2,1000.0,-23.0,85.0,...,?,MARRIED,BLACK/CARIBBEAN ISLAND,DIRECT OBSERVATION,0,4,42,4,SupplementalOxygen,2724
30004018,2159-01-08 12:00:00,None,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,...,ENGLISH,MARRIED,WHITE,EW EMER.,0,0,56,2,InvasiveVent,E9309
30009597,2175-10-06 17:00:00,CVVHDF,-30.0,150.0,150.0,-50.0,Prismasate K2,1000.0,3.0,148.0,...,ENGLISH,MARRIED,BLACK/AFRICAN AMERICAN,EW EMER.,0,0,45,7,InvasiveVent,42832
30014281,2187-04-29 10:00:00,CVVHDF,-33.0,120.0,180.0,0.0,Prismasate K2,500.0,32.0,107.0,...,ENGLISH,SINGLE,UNABLE TO OBTAIN,ELECTIVE,0,10,55,3,SupplementalOxygen,1977
30031755,2124-04-24 06:00:00,CVVHDF,-38.0,150.0,0.0,200.0,Prismasate K4,500.0,-2.0,190.0,...,ENGLISH,WIDOWED,WHITE,URGENT,1,4,43,9,SupplementalOxygen,5849
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39957610,2159-04-10 13:00:00,None,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,...,ENGLISH,MARRIED,UNKNOWN,DIRECT OBSERVATION,0,3,49,5,InvasiveVent,43310
39961682,2155-01-16 19:00:00,CVVHDF,-59.0,200.0,220.0,0.0,Prismasate K4,700.0,0.0,136.0,...,ENGLISH,MARRIED,WHITE,EU OBSERVATION,0,0,53,1,InvasiveVent,2720
39965149,2147-12-12 12:32:00,None,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,...,ENGLISH,WIDOWED,WHITE,SURGICAL SAME DAY ADMISSION,0,5,77,4,InvasiveVent,99592
